In [3]:
#Importowanie bibliotek
import qutip as qt
import matplotlib.pyplot as plt
import numpy as np

#Deklaracja macierzy pauliego
#------TUTAJ BYŁ BŁĄD!-----------
sx = qt.sigmax()  #sigma x -> obraca wokol osi x
sy = qt.sigmay()  #sigma y -> obraca wokol osi y
sz = qt.sigmaz()  #sigma z -> obraca wokol osi z
s0 = qt.qeye(2)   #operator nic nie robiacy "jedynka"

In [4]:
#Bramki kwantowe
def Ry(psi,qubit,kat): #Obrot wokol osi y. Argumenty=(psi=stan na wejściu, psi = na którym kubicie, kat)
    times = np.linspace(0,kat,21)  #wektor czasów
    if qubit==0: #qubit a (moj)
        Ha = qt.tensor(sy/2,s0) #Hamiltonian obracajacy qubit a wokol osi y
        res = qt.mesolve(Ha,psi,times) #symulacja ewolucji od stanu psi przez czas "kat"
        psi = res.states[-1]   #pobranie ostatniego stanu
    if qubit==1: #sasiada b
        Hb = qt.tensor(s0,sy/2)#Hamiltonian obracajacy qubit b wokol osi y
        res = qt.mesolve(Hb,psi,times) #symulacja ewolucji od stanu psi przez czas "kat"
        psi = res.states[-1] #pobranie ostatniego stanu
    return psi  #zwrócenie stanu na wyjściu


def Rx(psi,qubit,kat): #Obrot wokol osi x. Argumenty=(psi=stan na wejściu, psi = na którym kubicie, kat)
    times = np.linspace(0,kat,21)   #wektor czasów
    if qubit==0:  #qubit a (moj)
        Ha = qt.tensor(sx/2,s0) #Hamiltonian obracajacy qubit a wokol osi x
        res = qt.mesolve(Ha,psi,times) #symulacja ewolucji od stanu psi przez czas "kat"
        psi = res.states[-1] #pobranie ostatniego stanu
    if qubit==1: #sasiada b
        Hb = qt.tensor(s0,sx/2) #Hamiltonian obracajacy qubit b wokol osi x
        res = qt.mesolve(Hb,psi,times) #symulacja ewolucji od stanu psi przez czas "kat"
        psi = res.states[-1] #pobranie ostatniego stanu
    return psi #zwrócenie stanu na wyjściu



def Rzz(psi,kat): #Oddziaływanie kubitów typu ZZ. Argumenty=(psi=stan na wejściu, kat=watunkowy kat obrotu)
    times = np.linspace(0,kat,21)   #wektor czasów 
    Hzz = qt.tensor(sz,sz)  #Hamiltonian H = sigmaz_a sigmaz_b, delta= 1 tak żeby kat wyrazony byl w czasie
    res = qt.mesolve(Hzz/2.,psi,times) #symulacja ewolucji wywolanej oddziaływaniem
    psi = res.states[-1] #pobranie ostatniego stanu
    return psi #zwrócenie stanu na wyjściu

Krok 1

In [5]:
#słownik: 0--> a,  1---> b
plus = qt.basis(2,0) #stan |+>
psi1 = qt.tensor(plus,plus)#|psi0> = |+a>|+b>

Krok 2

In [6]:
psi2 = Ry(psi1,0,np.pi/2)  #obraca kubit a(0) wokol osi y o pi/2
psi2 = Ry(psi2,1,np.pi/2)  #obraca kubit b(1) wokol osi y o pi/2

Krok 3 (oddiaływanie)

In [7]:
psi3 = Rzz(psi2,np.pi/2)  #Odziałwanie qubitów a i b przez czas t = hbar pi/2/delta (splatanie jest tutaj)

Krok 4(naprawa)

In [8]:
psi4 = Rx(psi3,1,np.pi/2)  #obrot kubitu b(1) wokol osi y o pi/2 (naprawiam baze zeby lepiej wygladala)

Krok 5 (sprawdzamy)

In [10]:
m0 = qt.tensor(sz,s0)    #Operator mierzący rzut na os z na kubicie a i nic nie robiacy na b
m1 = qt.tensor(s0,sz)    #Operator mierzący rzut na os z na kubicie b i nic nie robiacy na a
mzz = qt.tensor(sz,sz)   #Operator mierzący rzut na os z qubitu a i mnożący przez rzut na os a qubitu b

ez0 = qt.expect(m0,psi4)   #Ez_a = p_+a - p_-a czyli pstwo zmierzenia |+> - p-stwo zmierzenia |-> na kubicie a
ez1 = qt.expect(m1,psi4)   #Ez_a = p_+a - p_-a czyli pstwo zmierzenia |+> - p-stwo zmierzenia |-> na kubicie a
ezz = qt.expect(mzz,psi4) #Ezz_a = p(za*zb=1) - p(za*ab=-1) czyli pstwo zmierzenie (1,1) lub (-1,-1)
# minus pstwo zmierzenia (1,-1),(-1,1)

In [11]:
# przypadku splatania powinnismy miec ez0=0, ez1=0, ezz=1 
#pomiary na kazdym z kubitow sa losow (1,-1,-1,1 etc) ale ta losowosc jest skorelwana (1,1),(-1,-1) etc...

In [14]:
print("E_{za}="+str(ez0))
print("E_{zb}="+str(ez1))
print("E_{zz}="+str(ezz))

E_{za}=5.3403694347808894e-06
E_{zb}=5.340384972574164e-06
E_{zz}=0.999999999960775
